<a href="https://colab.research.google.com/github/thisishasan/prog_mod_b/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!wget -nc 'https://raw.githubusercontent.com/thisishasan/prog_mod_b/refs/heads/main/data.csv'

--2025-05-13 19:01:09--  https://raw.githubusercontent.com/thisishasan/prog_mod_b/refs/heads/main/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1644858 (1.6M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   1.57M  --.-KB/s    in 0.05s   

2025-05-13 19:01:09 (31.9 MB/s) - ‘data.csv’ saved [1644858/1644858]



In [7]:
import pandas as pd

# Replace 'your_file.csv' with the path to your actual CSV file
df = pd.read_csv('data.csv')

# Display the first 5 rows of the DataFrame
print(df.head())

  Principio Attivo                          Descrizione Gruppo  \
0        Fenitoina         FENITOINA 100MG 30 UNITA' USO ORALE   
1     Acido folico  ACIDO FOLICO 15MG 5 UNITA' USO PARENTERALE   
2     Acido folico        ACIDO FOLICO 5MG 20 UNITA' USO ORALE   
3     Acido folico        ACIDO FOLICO 5MG 60 UNITA' USO ORALE   
4     Acido folico        ACIDO FOLICO 5MG 28 UNITA' USO ORALE   

     Denominazione e Confezione   Titolare AIC      AIC  \
0    DINTOINA*30 cpr riv 100 mg  RECORDATI SpA  2129017   
1  FOLINA*5 fiale IM 15 mg 2 ml   TEOFARMA Srl  2309033   
2      FOLINA*20 cps molli 5 mg   TEOFARMA Srl  2309045   
3      FOLINA*60 cps molli 5 mg   TEOFARMA Srl  2309058   
4      FOLINA*28 cps molli 5 mg   TEOFARMA Srl  2309060   

  Codice Gruppo Equivalenza  
0                       LTA  
1                       H7D  
2                       H7B  
3                       H7E  
4                       H7C  


In [64]:
!pip install pymupdf

In [71]:
import requests
import fitz  # PyMuPDF
import re

# Iterate over each row and get structured data from the API
for index, row in df.iterrows():
    aic = row['AIC']

    # Build the API URL
    url = f"https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/formadosaggio/ricerca?query={aic}&spellingCorrection=true&page=0"

    # Make the GET request
    response = requests.get(url)

    # Check for successful response
    if response.status_code == 200:
        json_data = response.json()  # Parse the response as JSON

        # Access the nested data field
        content = json_data.get('data', {}).get('content', [])

        # Print each item in the content list
        for item in content:
            id = item.get('id')
            codice_atc = item.get('codiceAtc')
            descrizione_atc = item.get('descrizioneAtc')
            codice_sis = item.get('medicinale').get('codiceSis')
            aic6 = item.get('medicinale').get('aic6')
            leaflet_url = f"https://medicinali.aifa.gov.it/it/#/it/organizzazione/{codice_sis}/farmaci/{aic6}/stampati/FI"
            pdf_url = f"https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/organizzazione/{codice_sis}/farmaci/{aic6}/stampati?ts=RCP"
            print(f"\n--- codiceAtc: {codice_atc} ---")
            print(f"\n--- descrizioneAtc: {descrizione_atc} ---")
            print(f"\n--- codiceSis: {codice_sis} ---")
            print(f"\n--- id: {id} ---")
            print(f"\n--- leafletUrl: {leaflet_url} ---")
            print(f"\n--- pdf_url: {pdf_url} ---")
            # Download the PDF
            response = requests.get(pdf_url)
            if response.status_code == 200:
                # Load and read the PDF
                pdf_path = 'rcp_downloaded.pdf'
                doc = fitz.open(pdf_path)

                # Combine all page text into one string
                full_text = ""
                for page in doc:
                    full_text += page.get_text()
                doc.close()

                # Normalize the text: remove line breaks that interrupt headings
                text = re.sub(r'(\n+)', '\n', full_text)  # Clean multiple newlines
                text = re.sub(r'([A-Za-z])\n([a-z])', r'\1 \2', text)  # Join words split over lines

                # List of sections with partial keywords (for fuzzy matching)
                sections = {
                    "4.1 Indicazioni terapeutiche": "4.1",
                    "4.2 Posologia e modo di somministrazione": "4.2",
                    "4.3 Controindicazioni": "4.3",
                    "4.4 Avvertenze speciali e precauzioni di impiego": "4.4",
                    "4.5 Interazioni con altri medicinali": "4.5",
                    "4.6 Fertilità, gravidanza e allattamento": "4.6",
                    "4.7 Effetti sulla capacità di guidare": "4.7",
                    "4.8 Effetti indesiderati": "4.8",
                    "4.9 Sovradosaggio": "4.9",
                    "6.2 Incompatibilità": "6.2"
                }

                # Create regex pattern for each heading
                pattern = '|'.join([f"{re.escape(val)}[^\n]*" for val in sections.values()])

                # Find matches with start and end positions
                matches = list(re.finditer(pattern, text, flags=re.IGNORECASE))

                # Extract and print content between matched sections
                extracted = {}
                for i in range(len(matches)):
                    title_line = matches[i].group().strip()
                    title_key = next((k for k, v in sections.items() if v in title_line), title_line)
                    start = matches[i].end()
                    end = matches[i + 1].start() if i + 1 < len(matches) else len(text)
                    body = text[start:end].strip()
                    extracted[title_key] = body

                # Print extracted sections
                for title, content in extracted.items():
                    print(f"\n{'='*80}\n{title}\n{'='*80}\n{content[:1000]}...\n")

            else:
                print(f"Failed to download PDF. Status code: {response.status_code}")
    else:
        print(f"Request failed for AIC {aic}: Status Code {response.status_code}")
    break



--- codiceAtc: ['N05AG02'] ---

--- descrizioneAtc: ['PIMOZIDE'] ---

--- codiceSis: 4127 ---

--- id: 0000009838 ---

--- leafletUrl: https://medicinali.aifa.gov.it/it/#/it/organizzazione/4127/farmaci/22907/stampati/FI ---

--- pdf_url: https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/organizzazione/4127/farmaci/22907/stampati?ts=RCP ---

4.1 Indicazioni terapeutiche
Indicazioni terapeutiche 
 
ACARBOSIO DOC Generici è raccomandato per il trattamento del diabete di tipo II (non insulino 
dipendente) nei pazienti non adeguatamente controllati mediante la sola dieta o l'associazione di 
dieta e (i) metformina e/o (ii) una sulfonilurea....


4.2 Posologia e modo di somministrazione
Posologia e modo di somministrazione 
 
Posologia 
Popolazione pediatrica 
Bambini e adolescenti di età inferiore a 18 anni 
L’efficacia e la sicurezza di acarbosio nei bambini e negli adolescenti non sono state stabilite. 
ACARBOSIO DOC Generici non è raccomandato nei pazienti di età inferiore a 18 anni. 
 
Adu